In [6]:
import torch
import json
import os
import numpy as np
import yaml
from collections import OrderedDict
from st_gcn.net.st_gcn import Model as Model

def load_skeleton_data(json_path, num_person_in=1, num_person_out=1):
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    skeleton_data = np.zeros((3, 150, 18, num_person_out))
    
    for frame in data['data']:
        frame_index = frame['frame_index']
        if frame_index >= 150:
            break
        
        for m, skeleton in enumerate(frame['skeleton'][:num_person_in]):
            if m >= num_person_out:
                break
            pose = np.array(skeleton['pose']).reshape(-1, 2)
            score = np.array(skeleton['score'])
            
            skeleton_data[0, frame_index, :, m] = pose[:, 0]
            skeleton_data[1, frame_index, :, m] = pose[:, 1]
            skeleton_data[2, frame_index, :, m] = score
    
    skeleton_data[0:2] = skeleton_data[0:2] - 0.5
    skeleton_data[0][skeleton_data[2] == 0] = 0
    skeleton_data[1][skeleton_data[2] == 0] = 0
    
    return skeleton_data

def predict_action(model, skeleton_data):
    data = torch.FloatTensor(skeleton_data).unsqueeze(0)
    data = data.to(next(model.parameters()).device)
    
    with torch.no_grad():
        output = model(data)
    
    probabilities = torch.nn.functional.softmax(output, dim=1)
    return probabilities.cpu().numpy()[0]

config_path = "../config/st_gcn/kinetics-skeleton-from-rawdata/test.yaml"

with open(config_path, 'r') as f:
    arg = yaml.safe_load(f)

model_args = arg['model_args']
model = Model(**model_args)

weights_path = "../model/epoch180_model.pt"
weights = torch.load(weights_path)

new_weights = OrderedDict()
for k, v in weights.items():
    name = k.replace("module.", "") if "module." in k else k
    new_weights[name] = v

model.load_state_dict(new_weights, strict=False)
model.eval()

device = torch.device(f"cuda:{arg['device']}" if torch.cuda.is_available() else "cpu")
model = model.to(device)

class_labels = ['walk', 'stand', 'sit', 'armExe', 'lieDown', 'fall']

json_dir = "../data/test"

for json_file in os.listdir(json_dir):
    if json_file.endswith('.json'):
        json_path = os.path.join(json_dir, json_file)
        
        skeleton_data = load_skeleton_data(json_path, num_person_in=model_args['num_person'], num_person_out=model_args['num_person'])
        
        prediction = predict_action(model, skeleton_data)
        predicted_class = np.argmax(prediction)
        
        with open(json_path, 'r') as f:
            data = json.load(f)
        actual_label_index = data.get('label_index', -1)  # 'label_index'를 사용
        actual_label = class_labels[actual_label_index] if 0 <= actual_label_index < len(class_labels) else 'Unknown'
        
        print(f"File: {json_file}")
        print(f"Actual label: {actual_label} (index: {actual_label_index})")
        print(f"Prediction probabilities: {prediction}")
        print(f"Predicted class index: {predicted_class}")
        print(f"Predicted action: {class_labels[predicted_class]}")
        print("--------------------")

print("All predictions completed.")

C:\Users\sh\AppData\Local\Temp\ipykernel_25140\1162874293.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load(weights_path)


File: armExe (1).json
Actual label: armExe (index: 3)
Prediction probabilities: [4.2292796e-04 2.9421135e-04 1.4901020e-04 9.9895048e-01 1.7103991e-04
 1.2278555e-05]
Predicted class index: 3
Predicted action: armExe
--------------------
File: armExe (2).json
Actual label: armExe (index: 3)
Prediction probabilities: [9.5394224e-02 8.3307438e-03 6.4069770e-02 8.3211643e-01 3.9899819e-06
 8.4800005e-05]
Predicted class index: 3
Predicted action: armExe
--------------------
File: armExe (3).json
Actual label: armExe (index: 3)
Prediction probabilities: [2.1490989e-04 3.3251534e-04 2.6510813e-06 9.9944967e-01 4.1371284e-09
 1.9868892e-07]
Predicted class index: 3
Predicted action: armExe
--------------------
File: armExe (4).json
Actual label: armExe (index: 3)
Prediction probabilities: [2.1322542e-04 1.1195191e-04 1.2669406e-06 9.9967349e-01 1.6379847e-08
 6.3287196e-08]
Predicted class index: 3
Predicted action: armExe
--------------------
File: armExe (5).json
Actual label: armExe (inde